# FinBERT for Sentiment Analysis

In [ ]:
!pip install tensorflow_text tf-models-official
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
import re
from bs4 import BeautifulSoup
import os
import shutil
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import matplotlib.pyplot as plt
from official.nlp import optimization
import tqdm
import pandas as pd

tf.get_logger().setLevel('ERROR')

In [6]:
# create a tokenizer object
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

# fetch the pretrained model
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [15]:
# A headline to be used as input
headline = "inflation is 10% up"

# Pre-process input phrase
input = tokenizer(headline, padding = True, truncation = True, return_tensors='pt')
# Run inference on the tokenized phrase
output = model(**input)

# Pass model output logits through a softmax layer.
sentim_scores = torch.nn.functional.softmax(output.logits, dim=-1)

In [16]:
sentim_scores

tensor([[0.1187, 0.0248, 0.8566]], grad_fn=<SoftmaxBackward0>)

In [14]:
data = pd.read_csv("C:/Users/MB-PC/ML/Cardano_reddit_extraction_101.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/MB-PC/ML/Cardano_reddit_extraction_101.csv'

In [ ]:
data = data.rename(columns={'Body':'headline'})
#data["headline"] = data["Content"]
#data = data.drop(["Content","Unnamed: 0"],axis=1)

In [ ]:
data = data.dropna()

In [ ]:
data = data.drop(["Unnamed: 0"],axis=1)
data
#data['headline'] = data['headline'].astype(str)

In [ ]:
if 'headline' in data:
    print('true')
else: print('false')

In [ ]:

part_data = data.head(4)
#data.dtypes

In [ ]:
def sentim_analyzer(df, tokenizer, model):
    for i in range (0, len(df)) :
        try:
            headline = df.loc[i, 'headline']

        except:
            return print(' \'headline\' column might be missing from dataframe')
        # Pre-process input phrase
        input = tokenizer(headline, padding = True, truncation = True, return_tensors='pt')
        # Estimate output
        output = model(**input)
        # Pass model output logits through a softmax layer.
        predictions = torch.nn.functional.softmax(output.logits, dim=-1)
        df.loc[i, 'Positive'] = predictions[0][0].tolist()
        df.loc[i, 'Negative'] = predictions[0][1].tolist()
        df.loc[i, 'Neutral']  = predictions[0][2].tolist()
    # rearrange column order
    try:
        df = df[['date',  'headline', 'Positive', 'Negative', 'Neutral']]
    except:
        pass
    return df

In [ ]:
sentim_data = sentim_analyzer(data, tokenizer, model)

In [ ]:
sentim_data

In [ ]:
# sentiment Between 0 and 1
sentim_data["Score"] = (sentim_data["Positive"] + 0.5*sentim_data["Neutral"]) / (sentim_data["Positive"]+ sentim_data["Neutral"]+ sentim_data["Negative"])